# Task description
- Classify the speakers of given features.
- Main goal: Learn how to use transformer.
- Baselines:
  - Easy: Run sample code and know how to use transformer.
  - Medium: Know how to adjust parameters of transformer.
  - Strong: Construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer. 
  - Boss: Implement [Self-Attention Pooling](https://arxiv.org/pdf/2008.01077v1.pdf) & [Additive Margin Softmax](https://arxiv.org/pdf/1801.05599.pdf) to further boost the performance.

- Other links
  - Kaggle: [link](https://www.kaggle.com/t/ac77388c90204a4c8daebeddd40ff916)
  - Slide: [link](https://docs.google.com/presentation/d/1HLAj7UUIjZOycDe7DaVLSwJfXVd3bXPOyzSb6Zk3hYU/edit?usp=sharing)
  - Data: [link](https://github.com/MachineLearningHW/ML_HW4_Dataset)

# Download dataset
- Data is [here](https://github.com/MachineLearningHW/ML_HW4_Dataset)

In [1]:
"""
If the links below become inaccessible, please connect TAs.
"""

# !wget https://github.com/MachineLearningHW/ML_HW4_Dataset/raw/0.0.1/Dataset.tar.gz.partaa
# !wget https://github.com/MachineLearningHW/ML_HW4_Dataset/raw/0.0.1/Dataset.tar.gz.partab
# !wget https://github.com/MachineLearningHW/ML_HW4_Dataset/raw/0.0.1/Dataset.tar.gz.partac
# !wget https://github.com/MachineLearningHW/ML_HW4_Dataset/raw/0.0.1/Dataset.tar.gz.partad

# !cat Dataset.tar.gz.parta* > Dataset.tar.gz

# !tar zxvf Dataset.tar.gz

'\nIf the links below become inaccessible, please connect TAs.\n'

# Git Clone `conformer` packages to fix the bugs.

In [ ]:
!git clone https://github.com/sooftware/conformer
!cp conformer/conformer -r ./Conformer
!rm conformer -rf
!mv Conformer conformer

# Packages

In [2]:
# Numerical Operations
import random
import numpy as np
import math

# Reading/Writing Data
import os
import csv
import json
from pathlib import Path

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import torchvision.models as models

# Visualize the training information
import matplotlib.pyplot as plt

# Conformer
# Ref: https://github.com/sooftware/conformer
from conformer import Conformer

# Utility Functions

In [3]:
def same_seed(seed): 
    """ Fixes random number generator seeds for reproducibility. """
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(dataset, train_ratio, seed):
    """ Split provided training data into training set and validation set. """
    train_set_size = int(train_ratio * len(dataset))
    valid_set_size = len(dataset) - train_set_size
    train_set, valid_set = random_split(dataset, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    print(f"[INFO]:\nTraining size: {train_set_size}\nValidation size: {valid_set_size}")
    return train_set, valid_set

def print_model(model, dataloader):
    """
    Print the model architecture.
    Arguments:
        model: Your Neural Network model.
        dataloader: Training data loader.
    Returns:
        void: No return value.
    """
    # Get the input size of training data.
    input_size = None
    for X, y in dataloader:
        input_size = X.shape[1:]        # input_size = (C, H, W)
        break

    # Pass model and the input size as parameters.
    print(model)

# Data

## Dataset
- Original dataset is [Voxceleb2](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox2.html).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb2.
- We randomly select 600 speakers from Voxceleb2.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training. 
- The architecture of data directory  
  - data directory  
  |---- metadata.json  
  |---- testdata.json  
  |---- mapping.json  
  |---- uttr-{random string}.pt  

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary. 
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [4]:
class SpeakerDataset(Dataset):
    def __init__(self, data_dir, segment_len=128):
        self.data_dir = data_dir
        self.segment_len = segment_len

        # Load the mapping from speaker neme to their corresponding "id". 
        mapping_path = Path(data_dir) / "mapping.json"   # Concatenate the path.
        mapping = json.load(mapping_path.open())
        self.speaker2id = mapping["speaker2id"]

        # Load metadata of training data, get each speaker's utterances' info.
        metadata_path = Path(data_dir) / "metadata.json" # Concatenate the path.
        metadata = json.load(open(metadata_path))["speakers"]
        
        # Get the total number of speaker.
        self.speaker_num = len(metadata.keys())
        self.data = []                                   # Each element records a speaker's utterances list and its id(label). 
        for speaker in metadata.keys():
            for utterances in metadata[speaker]:
                self.data.append([utterances["feature_path"], self.speaker2id[speaker]])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        feat_path, speaker = self.data[index]
        # Load preprocessed mel-spectrogram.
        mel = torch.load(os.path.join(self.data_dir, feat_path))

        # Segmemt mel-spectrogram into "segment_len" frames.
        if len(mel) > self.segment_len:
            # Randomly get the starting point of the segment.
            start = random.randint(0, len(mel) - self.segment_len)
            # Get a segment with "segment_len" frames.
            mel = torch.FloatTensor(mel[start:start+self.segment_len])
        else:
            mel = torch.FloatTensor(mel)
        # Turn the speaker id into long for computing loss later.
        speaker = torch.FloatTensor([speaker]).long()
        return mel, speaker

    def get_speaker_number(self):
        return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.

In [5]:
def collate_batch(batch):
    """
    Arguments:
        batch: A list of tuples with (feature, label)
               feature -> mel-spectrogram.
               label   -> speaker id.
    """
    # Process features within a batch.
    """ Collate a batch of data. """
    mel, speaker = zip(*batch)
    # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
    # batch_first=True to see the first dimension as batch_size.
    mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
    # mel: (batch size, segment_len, n_mels) n_mels: 40(The dimention of mel-spectrogram)
    return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, train_ratio, seed, segment_len, batch_size, n_workers):
    """Generate dataloader"""
    dataset = SpeakerDataset(data_dir, segment_len)
    speaker_num = dataset.get_speaker_number()
    # Split dataset into training dataset and validation dataset
    trainset, validset = train_valid_split(dataset, train_ratio, seed)

    train_loader = DataLoader(
        trainset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=n_workers,
        pin_memory=True,
        collate_fn=collate_batch,
    )
    valid_loader = DataLoader(
        validset,
        batch_size=batch_size,
        num_workers=n_workers,
        drop_last=True,
        pin_memory=True,
        collate_fn=collate_batch,
    )

    return train_loader, valid_loader, speaker_num

# Model Template
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [6]:
class Classifier(nn.Module):
    def __init__(self, d_model=256, segment_len=128, n_spks=600, dropout=0.3, device='cuda'):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)
        """
        Arguments:
            num_classes            : Number of classification classes.
            input_dim              : Dimension of input vector.
            encoder_dim            : Dimension of conformer encoder.
            num_encoder_layers     : Number of conformer blocks.
            num_attention_heads    : Number of attention heads(default=8).
            feed_forward_dropout_p : Probability of feed forward module dropout(default=0.1).
            attention_dropout_p    : Probability of attention module dropout(default=0.1).
            conv_dropout_p         : Probability of conformer convolution module dropout(default=0.1).
            half_step_residual     : Flag indication whether to use half step residual or not(fefault=True).
        """
        self.conformer = Conformer(num_classes=d_model, 
                                   input_dim=d_model, 
                                   encoder_dim=512,
                                   num_encoder_layers=6).to(device)
        self.segment_len = segment_len
        # TODO:
        #   Change Transformer to Conformer.
        #   https://arxiv.org/abs/2005.08100
        """
        Arguments:
            d_model        : The number of expected features in the input (required).
            nhead          : The number of heads in the multiheadattention models (required).
            dim_feedforward: The dimension of the feedforward network model (default=2048).
            dropout        : The dropout value (default=0.1).
            activation     : The activation function of intermediate layer, relu or gelu (default=relu).
        """
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, dim_feedforward=256, nhead=8, dropout=dropout, batch_first=True,
        )

        """
        TransformerEncoder is stacked of TransformerEncoderLayer.
        Arguments:
            encoder_layer : An instance of the TransformerEncoderLayer() class (required).
            num_layers    : The number of sub-encoder-layers in the encoder (required).
            norm          : The layer normalization component (optional).
        """
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

        # Project the the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.BatchNorm1d(d_model),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(d_model, n_spks),
        )

    def forward(self, mels):
        """
        Arguments:
            mels: (batch size, segment_len, n_mels) n_mels: 40(The dimention of mel-spectrogram)
        Returns:
            out: (batch size, n_spks)
        """
        # out: (batch size, segment_len, d_model)
        out = self.prenet(mels)
        """ Conformer """
        # out: (batch size, out_len, d_model)
        out, out_len = self.conformer(out, self.segment_len)
        """ Self-attention """
        out = self.encoder(out)
        """ Mean pooling """
        # out: (batch size, d_model)
        stats = out.mean(dim=1)
        """ Prediction layer """
        out = self.pred_layer(stats)
        return out

# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [7]:
def get_cosine_schedule_with_warmup(
    optimizer: torch.optim.Optimizer,
    num_warmup_steps: int,
    num_training_steps: int,
    num_cycles: float = 0.5,
    last_epoch: int = -1,
):
    """
    Create a schedule with a learning rate that decreases following the values of the cosine function between the
    initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
    initial lr set in the optimizer.

    Args:
        optimizer (:class:`~torch.optim.Optimizer`):
        The optimizer for which to schedule the learning rate.
        num_warmup_steps (:obj:`int`):
        The number of steps for the warmup phase.
        num_training_steps (:obj:`int`):
        The total number of training steps.
        num_cycles (:obj:`float`, `optional`, defaults to 0.5):
        The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
        following a half-cosine).
        last_epoch (:obj:`int`, `optional`, defaults to -1):
        The index of the last epoch when resuming training.

    Return:
        :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
    """
    def lr_lambda(current_step):
        # Warmup
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        # decadence
        progress = float(current_step - num_warmup_steps) / float(
            max(1, num_training_steps - num_warmup_steps)
        )
        return max(
            0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
        )

    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [8]:
def model_fn(batch, model, criterion, device):
    """Forward a batch through the model."""

    mels, labels = batch
    mels = mels.to(device)
    labels = labels.to(device)

    outs = model(mels)
    
    loss = criterion(outs, labels)

    # Get the speaker id with highest probability.
    preds = outs.argmax(1)
    # Compute accuracy.
    accuracy = torch.mean((preds == labels).float())

    return loss, accuracy

# Validate
- Calculate accuracy of the validation set.

In [9]:
def valid(dataloader, model, criterion, device): 
    """Validate on validation set."""

    model.eval()
    running_loss = 0.0
    running_accuracy = 0.0
    pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

    for i, batch in enumerate(dataloader):
        with torch.no_grad():
            loss, accuracy = model_fn(batch, model, criterion, device)
            running_loss += loss.item()
            running_accuracy += accuracy.item()
        
        pbar.update(dataloader.batch_size)
        pbar.set_postfix(
            loss=f"{running_loss / (i+1):.2f}",
            accuracy=f"{running_accuracy / (i+1):.2f}",
        )

    pbar.close()
    model.train()

    return running_loss / len(dataloader),  running_accuracy / len(dataloader)

# Main function

In [10]:
def parse_args():
    """arguments"""
    config = {
        "seed": 5401314,
        "train_ratio": 0.9,
        "data_dir": "./Dataset",
        "segment_len": 256, 
        "save_path": "./models/model.ckpt",
        "batch_size": 32,
        "n_workers": 8,
        "valid_steps": 2000,
        "warmup_steps": 1000,
        "save_steps": 10000,
        "total_steps": 70000,
    }

    return config


def main(
    seed,
    train_ratio,
    data_dir,
    segment_len,
    save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
    save_steps,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    train_loader, valid_loader, speaker_num = get_dataloader(data_dir, train_ratio, seed, segment_len, batch_size, n_workers)
    train_iterator = iter(train_loader)
    print(f"[Info]: Finish loading data!",flush = True)

    model = Classifier(n_spks=speaker_num, segment_len=segment_len).to(device)
    # print(model, train_loader)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.001)
    # scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
    print(f"[Info]: Finish creating model!",flush = True)
    
    if not os.path.isdir('./models'):
        os.mkdir('./models')                    # Create directory of saving models.

    best_accuracy = -1.0
    best_state_dict = None

    train_batch_acc_record, train_batch_loss_record = [], []
    valid_acc_record, valid_loss_record = [], []
    
    pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    for step in range(total_steps):
        # Get data
        try:
            batch = next(train_iterator)
        except StopIteration:
            train_iterator = iter(train_loader)
            batch = next(train_iterator)

        loss, accuracy = model_fn(batch, model, criterion, device)
        batch_loss = loss.item()
        batch_accuracy = accuracy.item()

        # Record accuracy and loss of each epoch.
        train_batch_acc_record.append(batch_accuracy)
        train_batch_loss_record.append(batch_loss)
        
        # Updata model
        loss.backward()
        optimizer.step()
        # scheduler.step()
        optimizer.zero_grad()

        # Log
        pbar.update()
        pbar.set_postfix(
            loss=f"{batch_loss:.2f}",
            accuracy=f"{batch_accuracy:.2f}",
            step=step + 1,
        )

        # Do validation
        if (step + 1) % valid_steps == 0:
            pbar.close()

            valid_loss, valid_accuracy = valid(valid_loader, model, criterion, device)
            
            # Record accuracy and loss of each epoch.
            valid_acc_record.append(valid_accuracy)
            valid_loss_record.append(valid_loss)

            # keep the best model
            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                best_state_dict = model.state_dict()

            pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

        # Save the best model so far.
        if (step + 1) % save_steps == 0 and best_state_dict is not None:
            torch.save(best_state_dict, save_path)
            pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

    pbar.close()

    return train_batch_acc_record, train_batch_loss_record, valid_acc_record, valid_loss_record

# Run scripts 

In [ ]:
scripts = [f"classifier{i+1}.py" for i in range(6)]
for script in scripts:
    print(f"[Info]: {script}")
    %run {script}

# Inference

In [11]:
%run ensemble.py

[Info]: Finish loading data!
[Info]: Use cuda now!


100%|██████████| 8000/8000 [00:41<00:00, 191.37it/s]
